In [1]:
import base64
import hmac
import json
import time
import requests
import toml

from urllib.parse import urljoin

# params
base_url = 'https://www.iihvvs.com'
api_config = toml.load('../config/api.toml')
api_key = api_config['api_key']
secret_key = api_config['secret_key']
passphrase = api_config['passphrase']

In [2]:
def request(method, uri, params=None, body=None, headers=None, auth=False):
    """Initiate network request
   @param method: request method, GET / POST / DELETE / PUT
   @param uri: request uri
   @param params: dict, request query params
   @param body: dict, request body
   @param headers: request http header
   @param auth: boolean, add permission verification or not
   """
    if params:
        query = "&".join(
            ["{}={}".format(k, params[k]) for k in sorted(params.keys())]
        )
        uri += "?" + query
    url = urljoin(base_url, uri)

    if auth:
        timestamp = (
                str(time.time()).split(".")[0]
                + "."
                + str(time.time()).split(".")[1][:3]
        )
        if body:
            body = json.dumps(body)
        else:
            body = ""
        message = str(timestamp) + str.upper(method) + uri + str(body)
        mac = hmac.new(
            bytes(secret_key, encoding="utf8"),
            bytes(message, encoding="utf-8"),
            digestmod="sha256",
        )
        d = mac.digest()
        sign = base64.b64encode(d)

        if not headers:
            headers = {}
        headers["Content-Type"] = "application/json"
        headers["OK-ACCESS-KEY"] = api_key
        headers["OK-ACCESS-SIGN"] = sign
        headers["OK-ACCESS-TIMESTAMP"] = str(timestamp)
        headers["OK-ACCESS-PASSPHRASE"] = passphrase
    result = requests.request(
        method, url, data=body, headers=headers, timeout=10
    ).json()
    if result.get("code") and result.get("code") != "0":
        return None, result
    return result, None

In [3]:
def get_asset(currency):
    params = {"ccy": currency}
    result = request(
        "GET", "/api/v5/account/balance", params=params, auth=True
    )
    return result


get_asset('USDT')

({'code': '0',
  'data': [{'adjEq': '',
    'borrowFroz': '',
    'details': [{'availBal': '0.0000000099259858',
      'availEq': '0.0000000099259858',
      'borrowFroz': '',
      'cashBal': '0.0000000099259858',
      'ccy': 'USDT',
      'crossLiab': '',
      'disEq': '246.06192296036352',
      'eq': '246.10868361024947',
      'eqUsd': '246.06192296036352',
      'fixedBal': '0',
      'frozenBal': '246.10868360032347',
      'imr': '0',
      'interest': '',
      'isoEq': '0',
      'isoLiab': '',
      'isoUpl': '0',
      'liab': '',
      'maxLoan': '',
      'mgnRatio': '',
      'mmr': '0',
      'notionalLever': '0',
      'ordFrozen': '0',
      'rewardBal': '0',
      'smtSyncEq': '0',
      'spotInUseAmt': '',
      'spotIsoBal': '0',
      'stgyEq': '246.10868360032347',
      'twap': '0',
      'uTime': '1710217333606',
      'upl': '0',
      'uplLiab': ''}],
    'imr': '',
    'isoEq': '0',
    'mgnRatio': '',
    'mmr': '',
    'notionalUsd': '',
    'ordFroz': '

In [4]:
def get_grid_order_list(order_type):
    params = {"algoOrdType": order_type}
    result = request(
        "GET", "/api/v5/tradingBot/grid/orders-algo-pending", params=params, auth=True
    )
    return result


# get spot grid
result = get_grid_order_list(order_type='grid')
result[0]['data']

[{'actualLever': '',
  'algoClOrdId': '',
  'algoId': '687405759617044480',
  'algoOrdType': 'grid',
  'arbitrageNum': '20',
  'availEq': '',
  'basePos': False,
  'baseSz': '0',
  'cTime': '1710162304481',
  'cancelType': '0',
  'copyType': '0',
  'direction': '',
  'floatProfit': '-0.04177479656',
  'gridNum': '25',
  'gridProfit': '0.03523336016',
  'instFamily': '',
  'instId': 'BTC-USDT',
  'instType': 'SPOT',
  'investment': '23.46',
  'lever': '',
  'liqPx': '',
  'maxPx': '75000',
  'minPx': '68500',
  'ordFrozen': '',
  'pnlRatio': '-0.0002788336061382',
  'profitSharingRatio': '',
  'quoteSz': '23.46',
  'rebateTrans': [{'rebate': '0', 'rebateCcy': 'BTC'},
   {'rebate': '0', 'rebateCcy': 'USDT'}],
  'runType': '1',
  'slTriggerPx': '',
  'state': 'running',
  'stopType': '',
  'sz': '',
  'tag': '',
  'totalPnl': '-0.0065414364',
  'tpTriggerPx': '',
  'triggerParams': [{'triggerAction': 'start',
    'delaySeconds': '0',
    'triggerStrategy': 'instant',
    'triggerType': 'm

In [9]:
# get future grid
result = get_grid_order_list(order_type='contract_grid')
result[0]['data']

[{'actualLever': '0.5959708139789247',
  'algoClOrdId': '',
  'algoId': '687008144513568768',
  'algoOrdType': 'contract_grid',
  'arbitrageNum': '6',
  'availEq': '',
  'basePos': True,
  'baseSz': '',
  'cTime': '1710067505651',
  'cancelType': '0',
  'copyType': '0',
  'direction': 'long',
  'floatProfit': '-0.14782599740996',
  'gridNum': '20',
  'gridProfit': '0.585803',
  'instFamily': 'BNB-USDT',
  'instId': 'BNB-USDT-SWAP',
  'instType': 'SWAP',
  'investment': '30',
  'lever': '8',
  'liqPx': '',
  'maxPx': '570',
  'minPx': '380',
  'ordFrozen': '',
  'pnlRatio': '0.014599233419668',
  'profitSharingRatio': '',
  'quoteSz': '',
  'rebateTrans': [{'rebate': '0', 'rebateCcy': 'USDT'}],
  'runType': '1',
  'slRatio': '',
  'slTriggerPx': '',
  'state': 'running',
  'stopType': '1',
  'sz': '30',
  'tag': '',
  'totalPnl': '0.43797700259004',
  'tpRatio': '',
  'tpTriggerPx': '',
  'triggerParams': [{'triggerAction': 'start',
    'delaySeconds': '0',
    'triggerStrategy': 'insta

In [11]:
def get_grid_order_details(order_type, algo_id):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
    }
    result = request("GET", "/api/v5/tradingBot/grid/orders-algo-details", params=params, auth=True)
    return result


result = get_grid_order_details('grid', 687405759617044480)
result

({'code': '0',
  'data': [{'activeOrdNum': '25',
    'actualLever': '',
    'algoClOrdId': '',
    'algoId': '687405759617044480',
    'algoOrdType': 'grid',
    'annualizedRate': '0.6264852738229058',
    'arbitrageNum': '20',
    'availEq': '',
    'basePos': False,
    'baseSz': '0',
    'cTime': '1710162304481',
    'cancelType': '0',
    'copyType': '0',
    'curBaseSz': '0.0001572332',
    'curQuoteSz': '12.18782557016',
    'direction': '',
    'eq': '',
    'floatProfit': '-0.01973070192',
    'gridNum': '25',
    'gridProfit': '0.03523336016',
    'instFamily': '',
    'instId': 'BTC-USDT',
    'instType': 'SPOT',
    'investment': '23.46',
    'lever': '',
    'liqPx': '',
    'maxPx': '75000',
    'minPx': '68500',
    'ordFrozen': '',
    'perMaxProfitRate': '0.0017890329927007255562',
    'perMinProfitRate': '0.0014727722772276442034',
    'pnlRatio': '0.0006608123716965',
    'profit': '0.03523336',
    'profitSharingRatio': '',
    'quoteSz': '23.46',
    'rebateTrans': 

In [19]:
def get_grid_sub_orders(order_type, algo_id, order_status):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
        "type": order_status,
    }

    result = request("GET", "/api/v5/tradingBot/grid/sub-orders", params=params, auth=True)
    return result

result = get_grid_sub_orders('grid', 687405759617044480, "live")
len(result[0]['data'])
result[0]['data'][0]

{'accFillSz': '0',
 'algoClOrdId': '',
 'algoId': '687405759617044480',
 'algoOrdType': 'grid',
 'avgPx': '0',
 'cTime': '1710234827244',
 'ccy': '',
 'cilentOrdid': '',
 'ctVal': '',
 'fee': '0',
 'feeCcy': 'BTC',
 'groupId': '',
 'instId': 'BTC-USDT',
 'instType': 'SPOT',
 'lever': '0',
 'ordId': '687709942161543223',
 'ordType': 'limit',
 'pnl': '0',
 'posSide': 'net',
 'px': '7.162E+4',
 'rebate': '0',
 'rebateCcy': 'USDT',
 'side': 'buy',
 'state': 'live',
 'sz': '0.0000131',
 'tag': '',
 'tdMode': 'cash',
 'uTime': '1710234827244'}

In [23]:
def get_grid_positions(order_type, algo_id):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
    }

    result = request("GET", "/api/v5/tradingBot/grid/positions", params=params, auth=True)
    return result

result = get_grid_positions('contract_grid', '687008144513568768')
result

({'code': '0',
  'data': [{'adl': '1',
    'algoClOrdId': '',
    'algoId': '687008144513568768',
    'avgPx': '524.6291666666666668',
    'cTime': '1710067505657',
    'ccy': 'USDT',
    'imr': '2.6425',
    'instId': 'BNB-USDT-SWAP',
    'instType': 'SWAP',
    'last': '528.6',
    'lever': '8',
    'liqPx': '',
    'markPx': '528.5',
    'mgnMode': 'cross',
    'mgnRatio': '47.87387079685098',
    'mmr': '0.13741',
    'notionalUsd': '21.134080800000003',
    'pos': '4',
    'posSide': 'net',
    'uTime': '1710231186151',
    'upl': '0.1548333333333312',
    'uplRatio': '0.0590258198251146'}],
  'msg': ''},
 None)